## RDP Learning

In this notebook, we show how to perform PAC learning
over RDPs.

### Experiment 1: Rotating MAB

In [7]:
from functools import partial
from typing import Callable, Sequence
import gym
import numpy as np
from gym.wrappers import TimeLimit
from src import NonMarkovianRotatingMAB
from src.learn_pdfa.base import learn_pdfa
from src.learn_pdfa.common import Generator, PalmerParams, MultiprocessedGenerator
from src.learn_pdfa.learn_subgraph import learn_subgraph
from src.pdfa import PDFA
from src.pdfa.types import Word

env = NonMarkovianRotatingMAB(winning_probs=[0.9, 0.2])

print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

s = env.reset()
print(f"Initial state: {s}")

action = 0
sp, reward, _, _ = env.step(action)
print("-" * 10)
print(f"Action: {action}")
print(f"Next state: {sp}")
print(f"Reward: {reward}")

sp, reward, _, _ = env.step(action)
print("-" * 10)
print(f"Action: {action}")
print(f"Next state: {sp}")
print(f"Reward: {reward}")


Observation space: Discrete(2)
Action space: Discrete(2)
Initial state: 0
----------
Action: 0
Next state: 1
Reward: 1.0
----------
Action: 0
Next state: 1
Reward: 1.0


### Learning

In [8]:
class RDPGenerator(Generator):
    """Generate a trace against."""

    def __init__(self, env: gym.Env, policy: Callable, stop_probability: float = 0.05):
        self._env = env
        self._policy = policy
        self._stop_probability = stop_probability

        self.obs_space_dim = self._env.observation_space.n
        self.action_dim = self._env.action_space.n
        self.nb_rewards = 2  # TODO fix
        self._encoder = partial(np.ravel_multi_index, dims=(self.action_dim, self.nb_rewards, self.obs_space_dim))

    def alphabet_size(self) -> int:
        """Get the alphabet size."""
        return int(np.prod([self.action_dim, self.nb_rewards, self.obs_space_dim]))

    def sample(self, n: int = 1) -> Sequence[Word]:
        result = []
        for _ in range(n):
            word = self._sample_word()
            result.append(word)
        return result

    def _should_stop(self) -> bool:
        """Return True if the current episode should stop, false otherwise."""
        return np.random.random() < self._stop_probability

    def _sample_word(self) -> Word:
        """Sample one word."""
        initial_state = self._env.reset()
        done = False
        trace = [(0, 0, initial_state)]
        while not done:
            if self._should_stop():
                break
            action = self._policy()
            obs, reward, done, _ = self._env.step(action)
            trace += [(action, int(reward), obs)]

        trace = [self._encoder(x) for x in trace]
        return trace

In [9]:
stop_probability = 0.05
winning_probs = (1.0, 0.0)
optimal_avg_reward = max(winning_probs)
env = gym.make("NonMarkovianRotatingMAB-v0", winning_probs=winning_probs)
env = TimeLimit(env, max_episode_steps=4)

def exploration_policy(env: gym.Env):
    return env.action_space.sample()
policy = partial(exploration_policy, env)

rdp_generator = RDPGenerator(env, policy=policy, stop_probability=stop_probability)

In [10]:
examples = rdp_generator.sample(n=1000)

print(f"Apriori expected length of traces: 1/stop_prob = {1/stop_probability}")
print(f"Average length of traces: {np.mean([len(e) for e in examples])}")

Apriori expected length of traces: 1/stop_prob = 20.0
Average length of traces: 4.549


In [11]:
N = 300000
mp_rdp_generator = MultiprocessedGenerator(rdp_generator, nb_processes=8)
automaton = learn_pdfa(
    sample_generator=mp_rdp_generator,
    alphabet_size=rdp_generator.alphabet_size(),
    epsilon=0.4,
    delta_1=0.2,
    delta_2=0.2,
    mu=0.2,
    n=3,
    n1_max_debug=N,
    n2_max_debug=N,
    m0_max_debug=10000
)

[2020-10-30 20:30:33,352][src.learn_pdfa][INFO] Parameters: ('PalmerParams(sample_generator=<src.learn_pdfa.common.MultiprocessedGenerator '
 'object at 0x7f63e6176890>, alphabet_size=8, epsilon=0.4, delta_1=0.2, '
 'delta_2=0.2, mu=0.2, n=3, m0_max_debug=10000, n1_max_debug=300000, '
 'n2_max_debug=300000)')
[2020-10-30 20:30:33,353][src.learn_pdfa][INFO] N1 = 616983.093395157, N2 = 57183600.0. Chosen: 57183600
[2020-10-30 20:30:33,353][src.learn_pdfa][INFO] m0 = 238265
[2020-10-30 20:30:33,354][src.learn_pdfa][INFO] N = 57183600
[2020-10-30 20:30:33,354][src.learn_pdfa][INFO] using m0 = 10000, N = 300000
[2020-10-30 20:30:54,817][src.learn_pdfa][INFO] Sampling done.
[2020-10-30 20:30:54,818][src.learn_pdfa][INFO] Number of samples: 300000.
[2020-10-30 20:30:54,840][src.learn_pdfa][INFO] Avg. length of samples: 4.536053333333333.
[2020-10-30 20:30:55,237][src.learn_pdfa][INFO] Iteration 0
[2020-10-30 20:30:56,310][src.learn_pdfa][INFO] Iteration 1
[2020-10-30 20:30:57,680][src.learn_p

In [6]:
# Learned graph (ignore the probabilities)
tr = {
 0: {0: (0, 0.3), 3: (1, 0.4), 4: (2, 0.3)},
 1: {0: (3, 0.5), 7: (4, 0.5)},
 2: {3: (3, 0.5), 4: (4, 0.5)},
 3: {0: (5, 0.5), 7: (6, 0.5)},
 4: {3: (5, 0.5), 4: (6, 0.5)},
 5: {0: (7, 0.5), 7: (7, 0.5)},
 6: {3: (7, 0.5), 4: (7, 0.5)}
}
automaton = PDFA(7, rdp_generator.alphabet_size(), tr)

AssertionError: The following states cannot reach the final state: {0, 1, 2, 3, 4, 5, 6, 7}